In [1]:
import os
import numpy as np
import pyvista as pv
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Extend.DataExchange import read_step_file
from OCC.Extend.ShapeFactory import get_oriented_boundingbox
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
pv.set_jupyter_backend('trame')  # Try interactive 3D again

In [2]:
# Load the STEP and extract 'solid' as before
from OCC.Extend.DataExchange import read_step_file
shape = read_step_file("0444-1.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [3]:
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib_Add
from OCC.Core.gp import gp_Pln, gp_Ax3, gp_Pnt, gp_Dir
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
import numpy as np

def compute_cross_section_area_occ(solid):
    # 1) Compute true volume centroid & long axis via inertia
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    cm = props.CentreOfMass()
    centroid = np.array([cm.X(), cm.Y(), cm.Z()])
    # inertia‐axis
    inertia = props.MatrixOfInertia()
    T = np.array([[inertia.Value(i,j) for j in (1,2,3)] for i in (1,2,3)])
    eigvals, eigvecs = np.linalg.eigh(T)
    long_idx = np.argmin(eigvals)
    long_dir = gp_Dir(*eigvecs[:, long_idx])

    # 2) Build infinite plane at centroid ⟂ long_axis
    plane = gp_Pln(gp_Ax3(gp_Pnt(*centroid), long_dir))

    # 3) Section the solid with that plane
    section = BRepAlgoAPI_Section(solid, plane)
    section.ComputePCurveOn1(True)
    section.Build()
    sect = section.Shape()

    # 4) Collect all edges into one wire
    wire_maker = BRepBuilderAPI_MakeWire()
    exp_edge = TopExp_Explorer(sect, TopAbs_EDGE)
    while exp_edge.More():
        wire_maker.Add(exp_edge.Current())
        exp_edge.Next()
    wire = wire_maker.Wire()

    # 5) Build a face from that wire
    face = BRepBuilderAPI_MakeFace(wire, True).Face()

    # 6) Compute exact face area
    props2 = GProp_GProps()
    brepgprop.SurfaceProperties(face, props2)
    area = props2.Mass()  # in mm²

    return area


In [4]:
import numpy as np
import pandas as pd
from IPython.display import display

from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Cylinder
from OCC.Core.TopoDS import topods
from OCC.Core.gp import gp_Dir, gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Display.SimpleGui import init_display
from OCC.Core.AIS import AIS_TextLabel
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB

# 1) Build centroid + local axes (L, F, W)
props = GProp_GProps(); brepgprop.VolumeProperties(solid, props)
cm = props.CentreOfMass()
centroid = np.array([cm.X(), cm.Y(), cm.Z()])

im = props.MatrixOfInertia()
T  = np.array([[im.Value(i,j) for j in (1,2,3)] for i in (1,2,3)])
ev, evec = np.linalg.eigh(T)
order = np.argsort(ev)  # [long, flange, web]
dirs = [gp_Dir(*evec[:,i]) for i in order]
L = np.array([dirs[0].X(), dirs[0].Y(), dirs[0].Z()])
F = np.array([dirs[1].X(), dirs[1].Y(), dirs[1].Z()])
# F = F / np.linalg.norm(F)
W = np.array([dirs[2].X(), dirs[2].Y(), dirs[2].Z()])

# 2) Compute half‐extents and origin
bary, half_extents, _ = get_oriented_boundingbox(solid)
spans = [2*h for h in half_extents]
length, width, thickness = sorted(spans, reverse=True)
origin = centroid - 0.5*length*L - 0.5*width*F - 0.5*thickness*W

# 3) Start the Qt viewer
viewer, start_viewer, _, _ = init_display()

# Beam in light gray:
gray = Quantity_Color(0.25, 0.25, 0.5, Quantity_TOC_RGB)
viewer.DisplayShape(solid, color=gray, transparency=0.5, update=False)

# 3) Classify & collect holes
tol_long = 0.7
tol_web  = 0.7
hole_data = []
face_id = 0

exp = TopExp_Explorer(solid, TopAbs_FACE)
while exp.More():
    face_id += 1
    face    = topods.Face(exp.Current())
    adaptor = BRepAdaptor_Surface(face, True)
    if adaptor.GetType() == GeomAbs_Cylinder:
        cyl = adaptor.Cylinder()
        xyzc = cyl.Axis().Direction().XYZ()
        g    = np.array([xyzc.X(), xyzc.Y(), xyzc.Z()])
        ng   = np.linalg.norm(g)

        # skip length holes
        if abs(np.dot(g, L))/(ng*np.linalg.norm(L)) > tol_long:
            exp.Next(); continue

        # centroid
        pf = GProp_GProps(); brepgprop.SurfaceProperties(face, pf)
        c  = pf.CentreOfMass()
        fc = np.array([c.X(), c.Y(), c.Z()])

        # classify V/O/U
        if abs(np.dot(g, W))/(ng*np.linalg.norm(W)) > tol_web:
            code, rgb = "V", (1.0, 0.0, 0.0)  # red
        else:
            side = np.dot(fc - centroid, F)
            if side > 0:
                code, rgb = "O", (0.0, 1.0, 0.0)  # green
            else:
                code, rgb = "U", (0.0, 0.0, 1.0)  # blue

        hole_data.append((face, code, rgb, fc))
        
    exp.Next()

# Align NC origin
a_half_L, a_half_F, a_half_W = half_extents
origin_nc1 = (
    centroid
  - a_half_L * L     # to the head cross‐section (H)
  - a_half_F * F     # to the outside flange face (O)
  + a_half_W * W     # now truly to the underside flange face (U)
)

# Build Hole Table
rows = []
for idx, (face, code, rgb, fc) in enumerate(hole_data, start=1):
    # 1) Diameter
    adaptor = BRepAdaptor_Surface(face, True)
    diam    = 2.0 * adaptor.Cylinder().Radius()

    # 2) Vector from NC1 origin to hole center
    v = fc - origin_nc1

    # 3) Conditional projection based on hole code
    #    V‐holes lie in the L–F plane; O/U‐holes in the L–W plane
    if code == "V":
        x = float(np.dot(v, L))   # along length
        y = float(np.dot(v, F))   # across flange
    else:
        x = float(np.dot(v, L))   # along length
        y = float(np.dot(v, W))   # through web (flange thickness)

    rows.append({
        "Hole #":        idx,
        "Code":          code,
        "Diameter (mm)": round(diam, 2),
        "X (mm)":        round(x, 0),
        "Y (mm)":        round(y, 0),
    })

# Build and display DataFrame
df_holes = pd.DataFrame(rows)
display(df_holes)

# Render each hole face & marker using Quantity_Color
for idx, (face, code, rgb, fc) in enumerate(hole_data, start=1):
    qcol = Quantity_Color(*rgb, Quantity_TOC_RGB)
    viewer.DisplayShape(face, color=qcol, update=False)
    
    # draw the sphere
    x, y, z = fc
    sph = BRepPrimAPI_MakeSphere(gp_Pnt(x, y, z), 5.0).Shape()
    viewer.DisplayShape(sph, color=qcol, update=False)

    # Add label to the sphere based on index (to match the table)
    label = AIS_TextLabel()
    label.SetText(f"{idx}")
    label.SetPosition(gp_Pnt(x, y, z))
    
    # black text for contrast
    label.SetColor(Quantity_Color(0.0, 0.0, 0.0, Quantity_TOC_RGB))

    viewer.Context.Display(label, False)

# Add the NC Origin
yellow = Quantity_Color(1.0, 1.0, 0.0, Quantity_TOC_RGB)
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(*origin_nc1), 10.0).Shape()  # 10 mm radius
viewer.DisplayShape(sphere, color=yellow, update=True)


# Display Qt5 viewer
viewer.FitAll()
viewer.View_Iso()
start_viewer()


pyqt5 backend - Qt version 5.15.2


,Hole #,Code,Diameter (mm),X (mm),Y (mm)
0,1,V,12.0,961.0,152.0
1,2,V,12.0,461.0,152.0
2,3,V,12.0,-139.0,152.0
3,4,V,12.0,-739.0,152.0
4,5,V,12.0,-739.0,52.0
5,6,V,12.0,-139.0,52.0
6,7,V,12.0,461.0,52.0
7,8,V,12.0,961.0,52.0
8,9,U,12.0,-614.0,-1088.0
9,10,O,12.0,-614.0,-1088.0
